In [9]:
import math
import re

priority = {'+': 0,
            '-': 0,
            '*': 1,
            '/': 1,
            'exp': 2,
            'abs': 2,
            'sin': 2,
            'cos': 2}

ufunc_set = set(['exp', 'abs', 'sin', 'cos'])
    
    
class nodes(object):
    def __init__(self, type, value, children=[]):
        if type is 'op' and not children:
            raise TypeError('an op node must have a nonempty children list')
        self.type = type
        self.value = value
        self.children = children
        
    def op_func(self):
        if self.type is not 'op':
            raise TypeError('%s node has no attr op_func' % self.type)
        if self.value == '+':
            return lambda x, y: x+y
        if self.value == '-':
            return lambda x, y: x-y
        if self.value == '*':
            return lambda x, y: x*y
        if self.value == '/':
            return lambda x, y: x/y
        if self.value == 'exp':
            return math.exp
        if self.value == 'abs':
            return abs
        if self.value == 'sin':
            return math.sin
        if self.value == 'cos':
            return math.cos
    # 计算节点的值，若有变量需通过关键字传入变量的值  
    def calc(self, **var):
        if self.type is 'const':
            return self.value
        elif self.type is 'var':
            return var[self.value]
        else:
            return self.op_func()(*tuple(i.calc(**var) for i in self.children))
    # 编译为LaTex公式
    def tolatex(self):
        if self.type is 'const':
            return str(self.value)
        if self.type is 'var':
            return r'\color{#0F8F9A}{%s}' % self.value
        elif self.type is 'op':
            ch_list = []
            for ch in self.children:              # 对括号显示与否的处理
                ch_list.append(ch.tolatex() if ch.type is 'const' or 
                               ch.type is 'var' or 
                               self.value == '/' or
                               self.value in ufunc_set or
                               priority[ch.value] >= priority[self.value]
                               else r'\left(%s\right)' % ch.tolatex())
            if self.value == '+':
                return r'%s + %s' % tuple(ch_list)
            if self.value == '-':
                return r'%s - %s' % tuple(ch_list)
            if self.value == '*':                 # 如果乘号右边为变量，左边为常数，则不显示乘号
                return (r'%s \cdot %s' % tuple(ch_list) if self.children[1].type != 'var'
                        else r'%s %s' % tuple(ch_list))
            if self.value == '/':
                return r'\frac{%s}{%s}' % tuple(ch_list)
            if self.value == 'exp':
                return r'\mathrm{e}^{%s}' % tuple(ch_list)
            if self.value == 'abs':
                return r'\left|%s\right|' % tuple(ch_list)
            if self.value == 'sin':
                return r'\mathrm{sin}\left(%s\right)' % tuple(ch_list)
            if self.value == 'cos':
                return r'\mathrm{cos}\left(%s\right)' % tuple(ch_list)
            
        
class ast(object):
    def __init__(nodes=[], lines=[]):
        pass
        
dig = re.compile('^\d+?$')
flo = re.compile('^\d+?\.\d+?$')
var_name = re.compile('^\w+?$')
func = re.compile('^(\w+?)\((.+?)\)$')

# 剖析
def parse(formula):
    #print(formula)

    # 括号匹配
    pare_stack = []
    pare_map = {}
    for i in enumerate(formula):
        if i[1] == '(':
            pare_stack.append(i)
        elif i[1] == ')':
            l = pare_stack.pop()
            pare_map[l[0]] = i[0]
    
    # 去掉表达式两端的括号
    if pare_map.get(0) == len(formula)-1:
        formula = formula[1: -1]
        pare_map = {k-1: v-1 for k, v in pare_map.items() if k != 0}
    
    # 整数、浮点和变量
    if dig.match(formula):
        return nodes('const', int(formula))
    if flo.match(formula):
        return nodes('const', float(formula))
    if var_name.match(formula):
        return nodes('var', formula)
    
    # 函数节点
    if func.match(formula):
        func_name, ch = re.findall(func, formula)[0]
        if pare_map[len(func_name)] == len(formula)-1:
            return nodes('op', func_name, [parse(ch)])
    
    # 双目运算符
    op_list=[]
    for i in enumerate(formula):
        if i[1] == '(':
            pare_stack.append(i)
        elif i[1] == ')':
            pare_stack.pop()
        # 忽略括号内
        if not pare_stack and i[1] in priority.keys():
            op_list.append(i)
    # 根据运算顺序排序
    op_list.sort(key=lambda n: priority[n[1]])
    print(op_list)
    pri = priority[op_list[0][1]]
    for i in range(len(op_list)):
        if priority[op_list[i][1]] != pri:
            ch = [parse(formula[:op_list[i-1][0]]),
                  parse(formula[op_list[i-1][0]+1:])]
            return nodes('op', op_list[i-1][1], ch)
    ch = [parse(formula[:op_list[-1][0]]),
          parse(formula[op_list[-1][0]+1:])]
    return nodes('op', op_list[-1][1], ch)

In [8]:
from IPython.display import Latex
formula = '(abs(math.exp(4+1)/7)+math.sin(4))/4+math.sin(4+7)'
s = parse('(x)')
print(s.calc(x=4, y=3))
print(eval(formula))
print(s.tolatex())
Latex(r'$%s$' % s.tolatex())

4
4.111279137571479
\color{#0F8F9A}{x}


<IPython.core.display.Latex object>